**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
FD_REG=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
ls -lh ${FD_REG}/${FN_REG}
cat ${FD_REG}/${FN_REG} | wc -l
echo +++++++++++++++++++++++++++++++++++
head ${FD_REG}/${FN_REG}
echo +++++++++++++++++++++++++++++++++++
tail ${FD_REG}/${FN_REG}

-rw-rw-r-- 1 kk319 reddylab 5.7M Oct 17 05:28 /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
247520
+++++++++++++++++++++++++++++++++++
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613
chr1	30803	31072
chr1	101603	101849
chr1	115411	115986
chr1	118518	118743
chr1	136071	137429
+++++++++++++++++++++++++++++++++++
chrX	155966947	155967395
chrX	155971561	155972273
chrX	155986274	155987619
chrX	155997267	155998045
chrX	156000382	156003205
chrX	156009687	156010227
chrX	156016391	156016836
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


In [3]:
ls ${FD_RES}/results

A001_K562_WSTARRseq  KS91_K562_ASTARRseq  Tewhey_K562_TileMPRA


In [5]:
PREFIX=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${PREFIX}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/*WGS*bed.gz | xargs -n 1 basename

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count
A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz


In [6]:
FP_INPS=($(ls ${FD_INP}/*WGS*bed.gz))

echo Total: ${#FP_INPS[@]}

Total: 7


In [15]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.%a.txt \
    --array 0-6 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output directory
PREFIX=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${PREFIX}/fragment_count
FD_OUT=${FD_RES}/results/${PREFIX}/coverage_astarrseq_peak_macs_input

FP_INPS=(${FD_INP}/*WGS*bed.gz)
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}

FD_REG=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
zcat  ${FP_INP} | head -n 3 
echo

echo "Input  file: " ${FD_REG}/${FN_REG}
echo
echo "Show the first few lines of the input file"
cat  ${FD_REG}/${FN_REG} | head -n 3 
echo
echo "Show the last few lines of the input file"
cat  ${FD_REG}/${FN_REG} | tail -n 3 
echo

### execute
bedtools map \
    -a ${FD_REG}/${FN_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FD_OUT}/${FN_OUT}

### Show I/O file
echo
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the output file"
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo
echo "Show the last few lines of the output file"
zcat ${FD_OUT}/${FN_OUT} | tail -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29423364


In [16]:
ls ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.*.txt

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.0.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.10.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.11.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.12.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.13.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.1.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.2.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.3.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.4.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_wstarr.5.

In [17]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.0.txt

Hostname:           x1-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-11-22+14:00:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show th

In [18]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_wstarr.4.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  4
Time Stamp:         12-11-22+14:00:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	14845	15207	chr1_14845_15207	1	.
chr1	15512	16248	chr1_15512_16248	1	.
chr1	15652	16254	chr1_15652_16254	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage_astarrseq_peak_macs_input/A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz

Show 